# Quickstart notebook
The example code here shows how to get up and running with Mosaic using the Python API.

In [0]:
from pyspark.sql.functions import *

## Enable Mosaic in the notebook
To get started, you'll need to attach the python library to your cluster and execute the `enable_mosaic` function.

In [0]:
from mosaic import enable_mosaic
enable_mosaic(spark, dbutils)

Mosaic has extra configuration options. Check the docs for more details.

In [0]:
help(enable_mosaic)

Help on function enable_mosaic in module mosaic.api.enable:

enable_mosaic(spark: pyspark.sql.session.SparkSession, dbutils=None) -> None
 Enable Mosaic functions.
 
 Use this function at the start of your workflow to ensure all of the required dependencies are installed and
 Mosaic is configured according to your needs.
 
 Parameters
 ----------
 spark : pyspark.sql.SparkSession
 The active SparkSession.
 dbutils : dbruntime.dbutils.DBUtils
 The dbutils object used for `display` and `displayHTML` functions.
 Optional, only applicable to Databricks users.
 
 Returns
 -------
 
 Notes
 -----
 Users can control various aspects of Mosaic's operation with the following Spark confs:
 
 - `spark.databricks.mosaic.jar.autoattach`: 'true' (default) or 'false'
 Automatically attach the Mosaic JAR to the Databricks cluster? (Optional)
 - `spark.databricks.mosaic.jar.location`
 Explicitly specify the path to the Mosaic JAR.
 (Optional and not required at all in a standard Databricks environment).
 - `spark.databricks.mosaic.geometry.api`: 'OGC' (default) or 'JTS'
 Explicitly specify the underlying geometry library to use for spatial operations. (Optional)
 - `spark.databricks.mosaic.index.system`: 'H3' (default)
 Explicitly specify the index system to use for optimized spatial joins. (Optional)

## Geometry constructors and the Mosaic internal geometry format

Mosaic allows users to create new Point geometries from a pair of Spark DoubleType columns.

In [0]:
from mosaic import st_point

lons = [-80., -80., -70., -70., -80.]
lats = [ 35.,  45.,  45.,  35.,  35.]

bounds_df = (
  spark
  .createDataFrame({"lon": lon, "lat": lat} for lon, lat in zip(lons, lats))
  .coalesce(1)
  .withColumn("point_geom", st_point("lon", "lat"))
)
bounds_df.show()

+----+-----+--------------------+
 lat| lon| point_geom|
+----+-----+--------------------+
35.0|-80.0|{1, [[[-80.0, 35....|
45.0|-80.0|{1, [[[-80.0, 45....|
45.0|-70.0|{1, [[[-70.0, 45....|
35.0|-70.0|{1, [[[-70.0, 35....|
35.0|-80.0|{1, [[[-80.0, 35....|
+----+-----+--------------------+

Mosaic Point geometries can be aggregated into LineString and Polygon geometries using the respective constructors.

In [0]:
from mosaic import st_makeline

bounds_df = (
  bounds_df
  .groupBy()
  .agg(collect_list("point_geom").alias("bounding_coords"))
  .select(st_makeline("bounding_coords").alias("bounding_ring"))
)
bounds_df.show()

+--------------------+
 bounding_ring|
+--------------------+
{3, [[[-80.0, 35....|
+--------------------+

In [0]:
from mosaic import st_makepolygon

bounds_df = bounds_df.select(st_makepolygon("bounding_ring").alias("bounds"))
bounds_df.show()

+--------------------+
 bounds|
+--------------------+
{5, [[[-80.0, 35....|
+--------------------+

## Geometry clipping without an index

Mosaic implements set intersection functions: contains, intersects, overlaps etc. Here you can see `st_contains` being used to clip points by a polygon geometry.

In [0]:
tripsTable = spark.table("delta.`/databricks-datasets/nyctaxi/tables/nyctaxi_yellow`")

In [0]:
from mosaic import st_contains
trips = (
  tripsTable
  .limit(5_000_000)
  .repartition(sc.defaultParallelism * 20)
  .drop("vendorId", "rateCodeId", "store_and_fwd_flag", "payment_type")
  .withColumn("pickup_geom", st_point("pickup_longitude", "pickup_latitude"))
  .withColumn("dropoff_geom", st_point("dropoff_longitude", "dropoff_latitude"))
  .crossJoin(bounds_df)
  .where(st_contains("bounds", "pickup_geom"))
  .where(st_contains("bounds", "dropoff_geom"))
  .cache()
)

In [0]:
trips.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+
 VTS|2010-04-15 13:41:00|2010-04-15 14:14:00| 2| 11.72| -74.01065| 40.710005| 1| -73.872435| 40.774478| 29.7| 0.0| 0.5| 0.0| 0.0| 30.2|{1, [[[-74.01065,...|{1, [[[-73.872435...|{5, [[[-80.0, 35....|
 VTS|2010-02-15 12:55:00|2010-02-15 13:09:00| 1| 5.23| -74.015962| 40.711412| 1| -73.984923| 40.774098| 14.5| 0.0| 0.5| 0.0| 0.0| 15.0|{1, [[[-74.015962...|{1, [[[-73.984923...|{5, [[[-80.0, 35....|
 CMT|2009-04-26 23:27:22|2009-04-26 23:40:26| 1| 1.8| -73.989036| 40.726695| null| -74.002777| 40.723206| 8.5| 0.0| null| 0.0| 0.0| 8.5|{1, [[[-73.989036...|{1, [[[-74.002777...|{5, [[[-80.0, 35....|
 VTS|2009-01-11 11:20:00|2009-01-11 11:26:00| 1| 0.97| -74.002512| 40.730425| null| -73.993673| 40.721282| 5.3| 0.0| null| 0.0| 0.0| 5.3|{1, [[[-74.002512...|{1, [[[-73.993673...|{5, [[[-80.0, 35....|
 CMT|2009-04-02 01:16:12|2009-04-02 01:23:59| 1| 2.2| -73.995599| 40.728161| null| -74.002376| 40.750366| 8.2| 0.0| null| 0.0| 0.0| 8.2|{1, [[[-73.995599...|{1, [[[-74.002376...|{5, [[[-80.0, 35....|
 VTS|2009-12-12 20:30:00|2009-12-12 20:40:00| 1| 0.53| -74.004212| 40.72939| null| -73.99607| 40.726298| 6.1| 0.5| 0.5| 0.0| 0.0| 7.1|{1, [[[-74.004212...|{1, [[[-73.99607,...|{5, [[[-80.0, 35....|
 CMT|2009-04-09 22:34:17|2009-04-09 22:46:21| 1| 2.9| -73.991842| 40.714915| null| -73.974893| 40.68876| 11.0| 0.0| null| 0.0| 0.0| 11.0|{1, [[[-73.991842...|{1, [[[-73.974893...|{5, [[[-80.0, 35....|
 VTS|2009-01-22 16:24:00|2009-01-22 16:35:00| 5| 3.34| -74.015607| 40.714728| null| -73.994955| 40.750188| 10.5| 1.0| null| 0.0| 0.0| 11.5|{1, [[[-74.015607...|{1, [[[-73.994955...|{5, [[[-80.0, 35....|
 VTS|2009-02-14 00:55:00|2009-02-14 01:05:00| 6| 2.16| -73.992295| 40.724642| null| -73.972718| 40.749897| 8.1| 0.5| null| 0.0| 0.0| 8.6|{1, [[[-73.992295...|{1, [[[-73.972718...|{5, [[[-80.0, 35....|
 VTS|2009-05-08 09:20:00|2009-05-08 09:25:00| 1| 0.92| -73.999583| 40.730202| null| -73.993888| 40.741373| 4.9| 0.0| null| 0.0| 0.0| 4.9|{1, [[[-73.999583...|{1, [[[-73.993888...|{5, [[[-80.0, 35....|
 VTS|2010-02-10 21:38:00|2010-02-10 21:45:00| 1| 1.97| -74.00303| 40.733112| 1| -74.011612| 40.707712| 6.9| 0.5| 0.5| 0.0| 0.0| 7.9|{1, [[[-74.00303,...|{1, [[[-74.011612...|{5, [[[-80.0, 35....|
 VTS|2009-02-06 22:26:00|2009-02-06 22:31:00| 1| 0.36| -73.991663| 40.73183| null| -73.987183| 40.73321| 4.5| 0.5| null| 0.0| 0.0| 5.0|{1, [[[-73.991663...|{1, [[[-73.987183...|{5, [[[-80.0, 35....|
 VTS|2009-12-27 01:37:00|2009-12-27 01:51:00| 3| 4.89| -73.99067| 40.734102| null| -73.943502| 40.784872| 12.9| 0.5| 0.5| 0.0| 0.0| 13.9|{1, [[[-73.99067,...|{1, [[[-73.943502...|{5, [[[-80.0, 35....|
 VTS|2009-12-17 19:28:00|2009-12-17 19:44:00| 5| 4.95| -74.000793| 40.727528| null| -73.980485| 40.783402| 14.5| 1.0| 0.5| 0.0| 0.0| 16.0|{1, [[[-74.000793...|{1, [[[-73.980485...|{5, [[[-80.0, 35....|
 VTS|2009-11-02 13:28:00|2009-11-02 13:53:00| 1| 5.29| -74.006522| 40.733205| null| -73.954813| 40.779498| 16.5| 0.0| 0.5| 3.3| 0.0| 20.3|{1, [[[-74.006522...|{1, [[[-73.954813...|{5, [[[-80.0, 35....|
 CMT|2009-02-20 16:02:08|2009-02-20 16:06:11| 2| 0.7| -73.998908| 40.713684| null| -74.011201| 40.713479| 5.1| 0.0| null| 0.0| 0.0| 5.1|{1, [[[-73.998908...|{1, [[[

## Read from GeoJson, compute some basic geometry attributes

You've seen how Mosaic can create geometries from Spark native data types but it also provides functions to translate Well Known Text (WKT), Well Known Binary (WKB) and GeoJSON representations to Mosaic geometries.

In [0]:
from mosaic import st_geomfromgeojson

geoJsonDF = (
  spark.read.format("json")
  .load("dbfs:/FileStore/shared_uploads/stuart.lynn@databricks.com/NYC_Taxi_Zones.geojson")
  .withColumn("geometry", st_geomfromgeojson(to_json(col("geometry"))))
  .select("properties.*", "geometry")
  .drop("shape_area", "shape_leng")
)

geoJsonDF.show()

+-------------+-----------+--------+--------------------+--------------------+
 borough|location_id|objectid| zone| geometry|
+-------------+-----------+--------+--------------------+--------------------+
 EWR| 1| 1| Newark Airport|{6, [[[-74.184452...|
 Queens| 2| 2| Jamaica Bay|{6, [[[-73.823375...|
 Bronx| 3| 3|Allerton/Pelham G...|{6, [[[-73.847926...|
 Manhattan| 4| 4| Alphabet City|{6, [[[-73.971774...|
Staten Island| 5| 5| Arden Heights|{6, [[[-74.174217...|
Staten Island| 6| 6|Arrochar/Fort Wad...|{6, [[[-74.063673...|
 Queens| 7| 7| Astoria|{6, [[[-73.904136...|
 Queens| 8| 8| Astoria Park|{6, [[[-73.923340...|
 Queens| 9| 9| Auburndale|{6, [[[-73.785024...|
 Manhattan| 24| 24| Bloomingdale|{6, [[[-73.959536...|
 Queens| 10| 10| Baisley Park|{6, [[[-73.783266...|
 Brooklyn| 11| 11| Bath Beach|{6, [[[-74.001098...|
 Manhattan| 12| 12| Battery Park|{6, [[[-74.015657...|
 Manhattan| 13| 13| Battery Park City|{6, [[[-74.012441...|
 Bronx| 18| 18| Bedford Park|{6, [[[-73.885139...|
 Brooklyn| 25| 25| Boerum Hill|{6, [[[-73.981552...|
 Brooklyn| 14| 14| Bay Ridge|{6, [[[-74.034073...|
 Queens| 15| 15|Bay Terrace/Fort ...|{6, [[[-73.777403...|
 Brooklyn| 22| 22| Bensonhurst West|{6, [[[-73.992549...|
Staten Island| 23| 23|Bloomfield/Emerso...|{6, [[[-74.195686...|
+-------------+-----------+--------+--------------------+--------------------+
only showing top 20 rows

Mosaic provides a number of functions for extracting the properties of geometries. Here are some that are relevant to Polygon geometries:

In [0]:
from mosaic import st_area, st_length
(
  geoJsonDF
  .withColumn("calculatedArea", abs(st_area("geometry")))
  .withColumn("calculatedLength", st_length("geometry"))
  .select("geometry", "calculatedArea", "calculatedLength")
).show()

+--------------------+--------------------+--------------------+
 geometry| calculatedArea| calculatedLength|
+--------------------+--------------------+--------------------+
{6, [[[-74.184452...|7.823067885002562E-4| 0.1163574531886787|
{6, [[[-73.823375...|0.001422779097814599| 0.8431218810128789|
{6, [[[-73.847926...|3.144141568206508E-4| 0.08434110590105784|
{6, [[[-73.971774...| 7.94539194214528E-5| 0.03559982116907461|
{6, [[[-74.174217...|4.979574893632193E-4| 0.09214648985735088|
{6, [[[-74.063673...|4.046076598541714E-4| 0.12905753305870837|
{6, [[[-73.904136...|3.897879892739018...| 0.10741717112348065|
{6, [[[-73.923340...|2.658771690499072E-5|0.027590691195158792|
{6, [[[-73.785024...|3.384438031986125...| 0.0997840924710119|
{6, [[[-73.959536...|4.193691000471793E-5|0.034022318280805826|
{6, [[[-73.783266...|4.358238180810105...| 0.09983947941552027|
{6, [[[-74.001098...|2.029913532381591E-4| 0.07243934539937844|
{6, [[[-74.015657...|1.078595394871048...| 0.01759380786230351|
{6, [[[-74.012441...|4.874002713725957E-5| 0.04726120195723124|
{6, [[[-73.885139...|1.488501639473639E-4| 0.06979954985700013|
{6, [[[-73.981552...|1.241682673558662...| 0.04714581993191898|
{6, [[[-74.034073...|6.636590898190788E-4| 0.13622530962992194|
{6, [[[-73.777403...|4.588438269978550...| 0.1295148763926726|
{6, [[[-73.992549...|4.613649657988681...| 0.11844001178496774|
{6, [[[-74.195686...|0.002090822778761513| 0.3007595308122533|
+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
geoJsonDF.count()

Out[12]: 263

## Example point-in-poly with indexing

Mosaic has built-in support for the popular spatial indexing library, H3. The user has access to functions for generating point indices and the sets of indices covering polygons, allowing point-in-polygon joins to be transformed into deterministic SQL joins.

In [0]:
from mosaic import point_index

trips_with_geom = (
  trips
  .withColumn("pickup_h3", point_index(lng="pickup_longitude", lat="pickup_latitude", resolution=lit(10)))
  .withColumn("dropoff_h3", point_index(lng="dropoff_longitude", lat="dropoff_latitude", resolution=lit(10)))
)

trips_with_geom.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds| pickup_h3| dropoff_h3|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+
 VTS|2010-04-15 13:41:00|2010-04-15 14:14:00| 2| 11.72| -74.01065| 40.710005| 1| -73.872435| 40.774478| 29.7| 0.0| 0.5| 0.0| 0.0| 30.2|{1, [[[-74.01065,...|{1, [[[-73.872435...|{5, [[[-80.0, 35....|622236750647558143|622236724015759359|
 VTS|2010-02-15 12:55:00|2010-02-15 13:09:00| 1| 5.23| -74.015962| 40.711412| 1| -73.984923| 40.774098| 14.5| 0.0| 0.5| 0.0| 0.0| 15.0|{1, [[[-74.015962...|{1, [[[-73.984923...|{5, [[[-80.0, 35....|622236750652506111|622236722237833215|
 CMT|2009-04-26 23:27:22|2009-04-26 23:40:26| 1| 1.8| -73.989036| 40.726695| null| -74.002777| 40.723206| 8.5| 0.0| null| 0.0| 0.0| 8.5|{1, [[[-73.989036...|{1, [[[-74.002777...|{5, [[[-80.0, 35....|622236723447431167|622236750705688575|
 VTS|2009-01-11 11:20:00|2009-01-11 11:26:00| 1| 0.97| -74.002512| 40.730425| null| -73.993673| 40.721282| 5.3| 0.0| null| 0.0| 0.0| 5.3|{1, [[[-74.002512...|{1, [[[-73.993673...|{5, [[[-80.0, 35....|622236750718369791|622236750716764159|
 CMT|2009-04-02 01:16:12|2009-04-02 01:23:59| 1| 2.2| -73.995599| 40.728161| null| -74.002376| 40.750366| 8.2| 0.0| null| 0.0| 0.0| 8.2|{1, [[[-73.995599...|{1, [[[-74.002376...|{5, [[[-80.0, 35....|622236750714699775|622236750591197183|
 VTS|2009-12-12 20:30:00|2009-12-12 20:40:00| 1| 0.53| -74.004212| 40.72939| null| -73.99607| 40.726298| 6.1| 0.5| 0.5| 0.0| 0.0| 7.1|{1, [[[-74.004212...|{1, [[[-73.99607,...|{5, [[[-80.0, 35....|622236750718238719|622236750714798079|
 CMT|2009-04-09 22:34:17|2009-04-09 22:46:21| 1| 2.9| -73.991842| 40.714915| null| -73.974893| 40.68876| 11.0| 0.0| null| 0.0| 0.0| 11.0|{1, [[[-73.991842...|{1, [[[-73.974893...|{5, [[[-80.0, 35....|622236750736654335|622236723565756415|
 VTS|2009-01-22 16:24:00|2009-01-22 16:35:00| 5| 3.34| -74.015607| 40.714728| null| -73.994955| 40.750188| 10.5| 1.0| null| 0.0| 0.0| 11.5|{1, [[[-74.015607...|{1, [[[-73.994955...|{5, [[[-80.0, 35....|622236750652145663|622236723439042559|
 VTS|2009-02-14 00:55:00|2009-02-14 01:05:00| 6| 2.16| -73.992295| 40.724642| null| -73.972718| 40.749897| 8.1| 0.5| null| 0.0| 0.0| 8.6|{1, [[[-73.992295...|{1, [[[-73.972718...|{5, [[[-80.0, 35....|622236750714372095|622236723495862271|
 VTS|2009-05-08 09:20:00|2009-05-08 09:25:00| 1| 0.92| -73.999583| 40.730202| null| -73.993888| 40.741373| 4.9| 0.0| null| 0.0| 0.0| 4.9|{1, [[[-73.999583...|{1, [[[-73.993888...|{5, [[[-80.0, 35....|622236750719549439|622236723430490111|
 VTS|2010-02-10 21:38:00|2010-02-10 21:45:00| 1| 1.97| -74.00303| 40.733112| 1| -74.011612| 40.707712| 6.9| 0.5| 0.5| 0.0| 0.0| 7.9|{1, [[[-74.00303,...|{1, [[[-74.011612...|{5, [[[-80.0, 35....|622236750719352831|622236750647754751|
 VTS|2009-02-06 22:26:00|2009-02-06 22:31:00| 1| 0.36| -73.991663| 40.73183| null| -73.987183| 40.73321| 4.5| 0.5| null| 0.0| 0.0| 5.0|{1, [[[-73.991663...|{1, [[[-73.987183...|{5, [[[-80.0, 35....|622236750715256831|622236723428655103|
 VTS|2009-12-27 01:37:00|2009-12-27 01:51:00| 3| 4.89| -73.99067| 40.734102| null| -73.943502| 40.784872| 12.9| 0.5| 0.5| 0.0| 0.0| 13.9|{1, [[[-73.99067,...|{1, [[[-73.943502...|{5, [[[-80.0, 35....|62223672342855

In [0]:
from mosaic import polyfill

neighbourhoods = (
  geoJsonDF
  .repartition(sc.defaultParallelism)
  .select("*", explode(polyfill("geometry", lit(10))).alias("h3"))
  .drop("geometry")
)

neighbourhoods.show()

+-------+-----------+--------+--------------------+------------------+
borough|location_id|objectid| zone| h3|
+-------+-----------+--------+--------------------+------------------+
 Queens| 207| 207|Saint Michaels Ce...|622236723991183359|
 Queens| 207| 207|Saint Michaels Ce...|622236723950878719|
 Queens| 207| 207|Saint Michaels Ce...|622236723989315583|
 Queens| 207| 207|Saint Michaels Ce...|622236723991216127|
 Queens| 207| 207|Saint Michaels Ce...|622236723988201471|
 Queens| 207| 207|Saint Michaels Ce...|622236723991248895|
 Queens| 207| 207|Saint Michaels Ce...|622236723990134783|
 Queens| 207| 207|Saint Michaels Ce...|622236723990167551|
 Queens| 207| 207|Saint Michaels Ce...|622236723950223359|
 Queens| 207| 207|Saint Michaels Ce...|622236723988299775|
 Queens| 207| 207|Saint Michaels Ce...|622236723951009791|
 Queens| 207| 207|Saint Michaels Ce...|622236723950256127|
 Queens| 207| 207|Saint Michaels Ce...|622236723988332543|
 Queens| 207| 207|Saint Michaels Ce...|622236723951042559|
 Queens| 207| 207|Saint Michaels Ce...|622236723991379967|
 Queens| 207| 207|Saint Michaels Ce...|622236723950682111|
 Queens| 207| 207|Saint Michaels Ce...|622236723950714879|
 Queens| 207| 207|Saint Michaels Ce...|622236723991052287|
 Queens| 207| 207|Saint Michaels Ce...|622236723950747647|
 Queens| 207| 207|Saint Michaels Ce...|622236723951140863|
+-------+-----------+--------+--------------------+------------------+
only showing top 20 rows

In [0]:
joined_df = trips_with_geom.alias("t").join(neighbourhoods.alias("n"), on=expr("t.pickup_h3 = n.h3"), how="inner")
joined_df.count()

Out[15]: 4944390

## Mosaic spatial join optimizations

Mosaic provides easy access to the optimized spatial join technique described in [this](https://databricks.com/blog/2021/10/11/efficient-point-in-polygon-joins-via-pyspark-and-bng-geospatial-indexing.html) blog post.

In [0]:
from mosaic import mosaic_explode

mosaic_neighbourhoods = (
  geoJsonDF
  .repartition(sc.defaultParallelism)
  .select("*", mosaic_explode("geometry", lit(10)))
  .drop("geometry")
)

mosaic_neighbourhoods.show()

+-------+-----------+--------+--------------------+-------+------------------+--------------------+
borough|location_id|objectid| zone|is_core| h3| wkb|
+-------+-----------+--------+--------------------+-------+------------------+--------------------+
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950682111| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950845951| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723951009791| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723988332543| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950780415| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991085055| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991248895| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950714879| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991183359| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723988299775| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723950747647| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991052287| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991216127| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723991379967| null|
 Queens| 207| 207|Saint Michaels Ce...| true|622236723951140863| null|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723990167551|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723991117823|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723950288895|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723988365311|[01 03 00 00 00 0...|
 Queens| 207| 207|Saint Michaels Ce...| false|622236723950878719|[01 03 00 00 00 0...|
+-------+-----------+--------+--------------------+-------+------------------+--------------------+
only showing top 20 rows

Mosaic also includes a convenience function for displaying dataframes with geometry columns.

In [0]:
from mosaic import displayMosaic
displayMosaic(mosaic_neighbourhoods)

borough location_id objectid zone is_core h3 wkb Queens 207 207 Saint Michaels Cemetery/Woodside true 622236723950682111 null Queens 207 207 Saint Michaels Cemetery/Woodside true 622236723950845951 null Queens 207 207 Saint Michaels Cemetery/Woodside true 622236723951009791 null Queens 207 207 Saint Michaels Cemetery/Woodside true 622236723988332543 null Queens 207 207 Saint Michaels Cemetery/Woodside true 622236723950780415 null Queens 207 207 Saint Michaels Cemetery/Woodside true 622236723991085055 null Queens 207 207 Saint Michaels Cemetery/Woodside true 622236723991248895 null Queens 207 207 Saint Michaels Cemetery/Woodside true 622236723950714879 null Queens 207 207 Saint Michaels Cemetery/Woodside true 622236723991183359 null Queens 207 207 Saint Michaels Cemetery/Woodside true 622236723988299775 null Queens 207 207 Saint Michaels Cemetery/Woodside true 622236723950747647 null Queens 207 207 Saint Michaels Cemetery/Woodside true 622236723991052287 null Queens 207 207 Saint Michaels Cemetery/Woodside true 622236723991216127 null Queens 207 207 Saint Michaels Cemetery/Woodside true 622236723991379967 null Queens 207 207 Saint Michaels Cemetery/Woodside true 622236723951140863 null Queens 207 207 Saint Michaels Cemetery/Woodside false 622236723990167551 AQMAAAABAAAACAAAAPY8dLGxeVLA6VrhpzthREBQEacmsHlSwCzwRvM7YURALjoqe655UsDQVHlCPGFEQDLNdJideVLAwM0A2T9hREA7QLKznHlSwFndcOJDYURALoYnBad5UsAQkCbYU2FEQK/88Ce0eVLAZyWQhk9hRED2PHQ= (truncated) Queens 207 207 Saint Michaels Cemetery/Woodside false 622236723991117823 AQMAAAABAAAABwAAAOIpeeS9eVLAJKnsu5lhRECzZeVqsnlSwBdxuoGdYURAOkktzK15UsCoCN1gsmFEQAhQ3x24eVLAs+u0VsJhREApsGIgxXlSwNEutQ++YURAmKTngL55UsA6lRlQnmFEQOIpeeS9eVLAJKnsu5lhREA= Queens 207 207 Saint Michaels Cemetery/Woodside false 622236723950288895 AQMAAAABAAAABAAAAHCPl/0weVLArMPMhAliRECYIBgKL3lSwGFYGa4KYkRAi7Ty7TJ5UsACk6eEDGJEQHCPl/0weVLArMPMhAliREA= Queens 207 207 Saint Michaels Cemetery/Woodside false 622236723988365311 AQMAAAABAAAABwAAAK/konTSeVLASvOUFwdiREB7GwmEw3lSwEDzCgEMYkRABTNK5b55UsBCbmzgIGJEQIMyuVTEeVLABkRlSCliREDXMW6d0XlSwDfZl+MKYkRAc1bS79J5UsBx6xfWB2JEQK/konTSeVLASvOUFwdiREA= Queens 207 207 Saint Michaels Cemetery/Woodside false 622236723950878719 AQMAAAABAAAABwAAACN2WWCMeVLA7xf/xgpiREBbBLJvfXlSwHmcLLAPYkRACRy10Hh5UsDOSXePJGJEQJwE45h5eVLAN+8YxSViRECGnYRtknlSwMIjagQuYkRAPYUaspZ5UsC8rxK9GmJEQCN2WWCMeVLA7xf/xgpiREA= Queens 207 207 Saint Michaels Cemetery/Woodside false 622236723989315583 AQMAAAABAAAACAAAAAUzSuW+eVLAQm5s4CBiREBc2Jz0r3lSwD6m0MklYkRAkWmsSax5UsDJAnFbNmJEQN8Iw8e6eVLALdeStjpiRECzKChPvXlSwNmLz2c3YkRArU6kh8N5UsAIEZ8dK2JEQIMyuVTEeVLABkRlSCliREAFM0o= (truncated) Queens 207 207 Saint Michaels Cemetery/Woodside false 622236723951828991 AQMAAAABAAAABAAAAAkctdB4eVLAzkl3jyRiREDThsRgd3lSwNxnZgglYkRAnATjmHl5UsA27xjFJWJEQAkctdB4eVLAzkl3jyRiREA= Queens 207 207 Saint Michaels Cemetery/Woodside false 622236723950157823 AQMAAAABAAAAEwAAAPVVafNCeVLAoiiAQu1hREAVy5baQnlSwFn1NcjtYURAeHRZaEF5UsDeb8Lq82FEQC3Aqh9AeVLAoMIXOvdhRED+jwPXPnlSwMsXuCX6YURAqd7fdT15UsCHZW/G/GFEQDC+guM7eVLA/UEgZ/9hREB11IA= (truncated) Queens 207 207 Saint Michaels Cemetery/Woodside false 622236723988234239 AQMAAAABAAAABgAAAPRMQ3/TeVLACJ3wYgJiRECv5KJ00nlSwErzlBcHYkRAc1bS79J5UsBx6xfWB2JEQNRiKSXTeVLA7S3oWgdiREBurLkT1HlSwEXQPSMFYkRA9ExDf9N5UsAInfBiAmJEQA== Queens 207 207 Saint Michaels Cemetery/Woodside false 622236723991445503 AQMAAAABAAAABwAAAKeMGcx5eVLA5bMP8mFhREDDQphhd3lSwMRFYx1pYURALcjEHXd5UsD5CAjoaWFEQGTGdzZyeVLAaKVKjnhhREAOEK2VdXlSwI09j3J3YURAcbiKNHp5UsC5yZ6TYmFEQKeMGcx5eVLA5bMP8mFhREA= Queens 207 207 Saint Michaels Cemetery/Woodside false 622236723990134783 AQMAAAABAAAABwAAAK788Ce0eVLAZyWQhk9hREAuhicFp3lSwBCQJthTYURA6PFzZqJ5UsACOh+3aGFEQNt3/beseVLAalLgrHhhREDKfrnxuHlSwJIY56d0YURArTdMVbZ5UsCbXDwXYWFEQK788Ce0eVLAZyWQhk9hREA= Queens 207 207 Saint Michaels Cemetery/Woodside false 622236723991314431 AQMAAAABAAAACwAAAM+QEpeKeVLAiU7Y4kNhREAdq/6CiXlSwBktgh1EYURA5nU2Cod5UsAelIKKRGFEQNgZIH2FeVLAxHfj6URhREDoPgSrg3lSwGnHhRdFYURABZyl93x5UsBH5uiJWGFEQKeMGcx5eVLA5rMP8

This also extends to plotting maps inside the notebook using the kepler.gl visualisation library using a notebook magic `%%mosaic_kepler`.

In [0]:
from mosaic import st_aswkt
(
  mosaic_neighbourhoods
  .select(st_aswkt(col("wkb")).alias("wkt"), col("h3"))
).createOrReplaceTempView("kepler_df")

In [0]:
%%mosaic_kepler
"kepler_df" "h3" "h3"

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter

<!doctype html> Kepler.gl

![mosaic kepler map example](../images/kepler-example.png)

Now the two datasets can be joined first on H3 index, with any false positives removed through a contains filter on a much simpler geometry.

In [0]:
mosaic_joined_df = (
  trips_with_geom.alias("t")
  .join(mosaic_neighbourhoods.alias("n"), on=expr("t.pickup_h3 = n.h3"), how="inner")
  .where(
    ~col("is_core") | 
    st_contains("wkb", "pickup_geom")
  )
)

mosaic_joined_df.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+---------+-----------+--------+--------------------+-------+------------------+--------------------+
vendor_id| pickup_datetime| dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|dropoff_longitude|dropoff_latitude|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount| pickup_geom| dropoff_geom| bounds| pickup_h3| dropoff_h3| borough|location_id|objectid| zone|is_core| h3| wkb|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+-----------------+----------------+-----------+-----+-------+----------+------------+------------+--------------------+--------------------+--------------------+------------------+------------------+---------+-----------+--------+--------------------+-------+------------------+--------------------+
 CMT|2009-04-09 22:34:17|2009-04-09 22:46:21| 1| 2.9| -73.991842| 40.714915| null| -73.974893| 40.68876| 11.0| 0.0| null| 0.0| 0.0| 11.0|{1, [[[-73.991842...|{1, [[[-73.974893...|{5, [[[-80.0, 35....|622236750736654335|622236723565756415|Manhattan| 148| 148| Lower East Side| false|622236750736654335|[01 03 00 00 00 0...|
 CMT|2009-04-09 22:34:17|2009-04-09 22:46:21| 1| 2.9| -73.991842| 40.714915| null| -73.974893| 40.68876| 11.0| 0.0| null| 0.0| 0.0| 11.0|{1, [[[-73.991842...|{1, [[[-73.974893...|{5, [[[-80.0, 35....|622236750736654335|622236723565756415|Manhattan| 45| 45| Chinatown| false|622236750736654335|[01 03 00 00 00 0...|
 CMT|2009-04-09 22:34:17|2009-04-09 22:46:21| 1| 2.9| -73.991842| 40.714915| null| -73.974893| 40.68876| 11.0| 0.0| null| 0.0| 0.0| 11.0|{1, [[[-73.991842...|{1, [[[-73.974893...|{5, [[[-80.0, 35....|622236750736654335|622236723565756415|Manhattan| 232| 232|Two Bridges/Sewar...| false|622236750736654335|[01 03 00 00 00 0...|
 VTS|2009-02-14 00:55:00|2009-02-14 01:05:00| 6| 2.16| -73.992295| 40.724642| null| -73.972718| 40.749897| 8.1| 0.5| null| 0.0| 0.0| 8.6|{1, [[[-73.992295...|{1, [[[-73.972718...|{5, [[[-80.0, 35....|622236750714372095|622236723495862271|Manhattan| 79| 79| East Village| false|622236750714372095|[01 03 00 00 00 0...|
 VTS|2009-02-14 00:55:00|2009-02-14 01:05:00| 6| 2.16| -73.992295| 40.724642| null| -73.972718| 40.749897| 8.1| 0.5| null| 0.0| 0.0| 8.6|{1, [[[-73.992295...|{1, [[[-73.972718...|{5, [[[-80.0, 35....|622236750714372095|622236723495862271|Manhattan| 114| 114|Greenwich Village...| false|622236750714372095|[01 03 00 00 00 0...|
 VTS|2009-12-27 01:37:00|2009-12-27 01:51:00| 3| 4.89| -73.99067| 40.734102| null| -73.943502| 40.784872| 12.9| 0.5| 0.5| 0.0| 0.0| 13.9|{1, [[[-73.99067,...|{1, [[[-73.943502...|{5, [[[-80.0, 35....|622236723428556799|622236723968606207|Manhattan| 234| 234| Union Sq| false|622236723428556799|[01 03 00 00 00 0...|
 VTS|2009-12-27 01:37:00|2009-12-27 01:51:00| 3| 4.89| -73.99067| 40.734102| null| -73.943502| 40.784872| 12.9| 0.5| 0.5| 0.0| 0.0| 13.9|{1, [[[-73.99067,...|{1, [[[-73.943502...|{5, [[[-80.0, 35....|622236723428556799|622236723968606207|Manhattan| 113| 113|Greenwich Village...| false|622236723428556799|[01 03 00 00 00 0...|
 VTS|2009-11-02 13:28:00|2009-11-02 13:53:00| 1| 5.29| -74.006522| 40.733205| null| -73.954813| 40.779498| 16.5| 0.0| 0.5| 3.3| 0.0| 20.3|{1, [[[-74.006522...|{1, [[[-73.954813...|{5, [[[-80.0, 35....|622236750719778815|622236722202443775|Manhattan| 249| 249| West Village| false|622236750719778815|[01 03 00 00 00 0...|
 VTS|2009-11-02 13:28:00|2009-11-02 13:53:00| 1| 5.29| -74.006522| 40.733205| null| -73.954813| 40.779498| 16.5| 0.0| 0.5| 3.3| 0.0| 20.3|{1, [[[-74.006522...|{1, [[[-73.954813...|{5, [[[-80.0, 35....|622236750719778815|622236722202443775|Ma